Щоб запустити наступні записники, якщо ви ще цього не зробили, вам потрібно створити обліковий запис на [AIMLAPI](https://aimlapi.com/) та отримати **API_KEY** і встановити ключ у файлі .env як `AIMLAPI_KEY`

In [9]:
%pip install openai python-dotenv

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:

import os
import pandas as pd
import requests
import numpy as np
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

API_KEY = os.getenv("AIMLAPI_KEY","")
assert API_KEY, "ERROR: AIMLAPI Key is missing"

client = OpenAI(
    api_key=API_KEY
    )

model = 'text-embedding-ada-002'

SIMILARITIES_RESULTS_THRESHOLD = 0.75
DATASET_NAME = "../embedding_index_3m.json"

⚠️ AIMLAPI_KEY не знайдено у файлі .env


Далі ми завантажимо індекс вбудовувань у Pandas DataFrame. Індекс вбудовувань зберігається в JSON-файлі під назвою `embedding_index_3m.json`. Індекс вбудовувань містить вбудовування для кожного транскрипту YouTube до кінця жовтня 2023 року.

In [3]:
def load_dataset(source: str) -> pd.core.frame.DataFrame:
    # Завантаження індексу відеосесій
    pd_vectors = pd.read_json(source)
    return pd_vectors.drop(columns=["text"], errors="ignore").fillna("")

Далі ми створимо функцію під назвою `get_videos`, яка шукатиме в індексі вбудовувань за запитом. Функція поверне 5 найрелевантніших відео, які найбільш схожі на запит. Функція працює наступним чином:

1. Спочатку створюється копія індексу вбудовувань.
2. Далі обчислюється вбудовування для запиту за допомогою API вбудовування OpenAI.
3. Потім в індексі вбудовувань створюється новий стовпчик під назвою `similarity`. Стовпчик `similarity` містить косинусну подібність між вбудовуванням запиту та вбудовуванням для кожного сегмента відео.
4. Далі індекс вбудовувань фільтрується за стовпчиком `similarity`. Індекс вбудовувань фільтрується, щоб включати тільки ті відео, які мають косинусну подібність більшу або рівну 0.75.
5. Нарешті, індекс вбудовувань сортується за стовпчиком `similarity`, і повертаються 5 найкращих відео.

In [4]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def get_videos(
    query: str, dataset: pd.core.frame.DataFrame, rows: int
) -> pd.core.frame.DataFrame:
    # створення копії набору даних
    video_vectors = dataset.copy()

    # отримання вбудовувань для запиту за допомогою прямого виклику API
    response = requests.post(
        "https://api.aimlapi.com/v1/embeddings",
        headers={
            "Authorization": f"Bearer {API_KEY}",
            "Content-Type": "application/json"
        },
        json={
            "model": model,
            "input": query,
            "encoding_format": "float"
        }
    )
    
    data = response.json()
    query_embeddings = data["data"][0]["embedding"]
    
    # створення нового стовпчика з обчисленою подібністю для кожного рядка
    video_vectors["similarity"] = video_vectors["ada_v2"].apply(
        lambda x: cosine_similarity(np.array(query_embeddings), np.array(x))
    )

    # повернення верхніх рядків
    return video_vectors.head(rows)

Ця функція дуже проста, вона просто виводить результати пошукового запиту.

In [5]:
def display_results(videos: pd.core.frame.DataFrame, query: str):
    def _gen_yt_url(video_id: str, seconds: int) -> str:
        """конвертація часу з формату 00:00:00 в секунди"""
        return f"https://youtu.be/{video_id}?t={seconds}"

    print(f"\nВідео, схожі на '{query}':")
    for _, row in videos.iterrows():
        youtube_url = _gen_yt_url(row["videoId"], row["seconds"])
        print(f" - {row['title']}")
        print(f"   Резюме: {' '.join(row['summary'].split()[:15])}...")
        print(f"   YouTube: {youtube_url}")
        print(f"   Схожість: {row['similarity']}")
        print(f"   Доповідачі: {row['speaker']}")

1. Спочатку індекс вбудовувань завантажується в DataFrame Pandas.
2. Далі користувачу пропонується ввести запит.
3. Потім викликається функція `get_videos` для пошуку в індексі вбудовувань за запитом.
4. Нарешті, викликається функція `display_results` для відображення результатів користувачу.
5. Потім користувачу пропонується ввести інший запит. Цей процес триває, доки користувач не введе `exit`.

![](../images/notebook-search.png?WT.mc_id=academic-105485-koreyst)

Вам буде запропоновано ввести запит. Введіть запит і натисніть Enter. Застосунок поверне список відео, які відповідають запиту. Застосунок також поверне посилання на місце у відео, де знаходиться відповідь на запитання.

Ось кілька запитів, які можна спробувати:

- Що таке Azure Machine Learning?
- Як працюють згорткові нейронні мережі?
- Що таке нейронна мережа?
- Чи можу я використовувати Jupyter Notebooks з Azure Machine Learning?
- Що таке ONNX?

In [6]:
pd_vectors = load_dataset(DATASET_NAME)

# отримання запиту користувача з вводу
while True:
    query = input("Введіть запит: ")
    if query == "exit":
        break
    videos = get_videos(query, pd_vectors, 5)
    display_results(videos, query)

NameError: name 'DATASET_NAME' is not defined

# Індивідуальне завдання
> Виконати пошук фрагментів, що містять "Computer Vision". Знайти найрелевантніший фрагмент за косинусною подібністю.

In [7]:
import os
import pandas as pd
import requests
import numpy as np
from dotenv import load_dotenv

# Завантажуємо ключі
load_dotenv()
API_KEY_SECURE = os.getenv("AIMLAPI_KEY", "")
assert API_KEY_SECURE, "Критична помилка: AIMLAPI_KEY не знайдено."

# Константи варіанту
TARGET_TOPIC = "Computer Vision"
EMBEDDING_MODEL = "text-embedding-ada-002"
FILE_PATH = "embedding_index_3m.json"

def calculate_cosine(vec1, vec2):
    """Обчислення косинусної схожості між двома векторами."""
    v1, v2 = np.array(vec1), np.array(vec2)
    norm_product = np.linalg.norm(v1) * np.linalg.norm(v2)
    return np.dot(v1, v2) / norm_product if norm_product > 0 else 0.0

def fetch_query_vector(text_input):
    """Отримання вбудовування через API з обробкою помилок."""
    headers = {
        "Authorization": f"Bearer {API_KEY_SECURE}",
        "Content-Type": "application/json"
    }
    payload = {
        "model": EMBEDDING_MODEL,
        "input": text_input
    }
    
    try:
        req = requests.post("https://api.aimlapi.com/v1/embeddings", 
                            headers=headers, json=payload, timeout=30)
        res_json = req.json()
        
        # Перевірка наявності даних у відповіді
        if "data" in res_json:
            return np.array(res_json["data"][0]["embedding"])
        else:
            print(f"Помилка API: {res_json}")
            return None
    except Exception as err:
        print(f"Помилка з'єднання: {err}")
        return None

def run_vision_search(df_source):
    """Пошук та аналіз фрагментів Computer Vision."""
    # 1. Фільтрація за ключовим словом у резюме
    matches = df_source[df_source["summary"].str.contains(TARGET_TOPIC, case=False, na=False)].copy()
    
    if matches.empty:
        print(f"Фрагментів за темою '{TARGET_TOPIC}' не виявлено.")
        return

    print(f"Знайдено {len(matches)} релевантних фрагментів. Починаємо розрахунок подібності...")

    # 2. Отримання вектора запиту
    target_vector = fetch_query_vector(TARGET_TOPIC)
    if target_vector is None: return

    # 3. Обчислення схожості для кожного знайденого рядка
    matches["similarity_score"] = matches["ada_v2"].apply(lambda x: calculate_cosine(target_vector, x))

    # 4. Визначення абсолютного лідера
    top_result = matches.sort_values("similarity_score", ascending=False).iloc[0]

    # 5. Красивий вивід результатів
    print("\n" + "="*50)
    print(f"РЕЗУЛЬТАТ ПОШУКУ ДЛЯ: '{TARGET_TOPIC}'")
    print("="*50)
    print(f"Відео: {top_result['title']}")
    print(f"Спікер: {top_result['speaker']}")
    print(f"Схожість: {top_result['similarity_score']:.4f}")
    print(f"Посилання: https://youtu.be/{top_result['videoId']}?t={int(top_result['seconds'])}")
    print(f"Короткий зміст: {top_result['summary'][:200]}...")
    print("="*50)

if __name__ == "__main__":
    # Завантаження даних
    data_frame = pd.read_json(FILE_PATH)
    # Очищуємо від зайвого тексту для економії пам'яті
    if "text" in data_frame.columns:
        data_frame = data_frame.drop(columns=["text"])
    
    run_vision_search(data_frame)

AssertionError: Критична помилка: AIMLAPI_KEY не знайдено.